In [ ]:
from DataUtil import *
import re
import numpy as np
from scipy.sparse import csr_matrix
import pickle

def remove_punctuation(sentence):
    # 定义一个正则表达式模式，用于匹配所有中英文标点符号
    pattern = r'[^\w\s]'
    # 使用 re.sub() 函数将匹配的标点符号替换为空字符串
    cleaned_sentence = re.sub(pattern, '', sentence)
    return cleaned_sentence

OPPOxiaobuReader = OPPOxiaobuBenchmarkReader('OPPO-xiaobu')
train_sentence_pairs, train_scores, dev_sentence_pairs, dev_scores = OPPOxiaobuReader.get_data()

# sentence_list = LCQMC_train_sentence_pairs+LCQMC_dev_sentence_pairs+LCQMC_test_sentence_pairs
sentence_list = train_sentence_pairs  + dev_sentence_pairs
def save_data_to_pkl(data, filename):
    with open(filename, 'wb') as file:
        pickle.dump(data, file)
    print(f"Data saved to {filename}")

def load_data_from_pkl(filename):
    with open(filename, 'rb') as file:
        data = pickle.load(file)
    return data



In [ ]:
from modelscope import snapshot_download
model_dir = snapshot_download('maidalun/bce-embedding-base_v1')

In [ ]:
# from BCEmbedding import EmbeddingModel
# 
# # list of sentences
# sentences = ['sentence_0', 'sentence_1']
# 
# # init embedding model
# model = EmbeddingModel(model_name_or_path="/data/.rootcache/modelscope/hub/maidalun/bce-embedding-base_v1")
# 
# # extract embeddings
# embeddings = model.encode(sentences)

In [ ]:
from gensim.models import KeyedVectors

# 替换成你的词库文件路径
file_path = 'tencent800w/tencent-ailab-embedding-zh-d200-v0.2.0/tencent-ailab-embedding-zh-d200-v0.2.0.txt'
# 加载词库
model = KeyedVectors.load_word2vec_format(file_path, binary=False, encoding='utf-8')

In [ ]:
import jieba
# jieba.enable_paddle()# 启动paddle模式。 0.40版之后开始支持，早期版本不支持
seg_list = list(jieba.cut("我手机丢了，我想换个手机", cut_all=True))

In [ ]:
seg_list

In [ ]:
from tqdm import tqdm
word_vector_length = 200
# 保证 n 为 50
n = 50
sentence_dict={}
for sentence1,sentence2 in tqdm(sentence_list):
    seg_list1 = list(jieba.cut(remove_punctuation(sentence1), cut_all=True))
    # 获取模型中每个词的向量
    sentence_embedding1 = []
    embeddings =[]
    for word in seg_list1:
        try:
            embeddings.append(model[word])
        except:
            embeddings.append(np.zeros(word_vector_length))
    if len(sentence_embedding1) > n:
        # 如果超过 50 个词，只取前 50 个词
        sentence_embedding1 = sentence_embedding1[:n]
    else:
        # 如果少于 50 个词，用零向量补充
        padding_length = n - len(sentence_embedding1)
        sentence_embedding1.extend([np.zeros(word_vector_length)] * padding_length)
    # 最终的句子向量矩阵为 50 x 200
    sentence_matrix1 = np.array(sentence_embedding1)
    sentence_matrix1 = csr_matrix(sentence_matrix1)
    sentence_dict[remove_punctuation(sentence1)] = sentence_matrix1
    
    seg_list2 = list(jieba.cut(remove_punctuation(sentence2), cut_all=True))
    # 获取模型中每个词的向量
    sentence_embedding2 = []
    embeddings =[]
    for word in seg_list2:
        try:
            embeddings.append(model[word])
        except:
            embeddings.append(np.zeros(word_vector_length))
    if len(sentence_embedding2) > n:
        # 如果超过 50 个词，只取前 50 个词
        sentence_embedding2 = sentence_embedding2[:n]
    else:
        # 如果少于 50 个词，用零向量补充
        padding_length = n - len(sentence_embedding2)
        sentence_embedding2.extend([np.zeros(word_vector_length)] * padding_length)
    # 最终的句子向量矩阵为 50 x 200
    sentence_matrix2 = np.array(sentence_embedding2)
    sentence_matrix2 = csr_matrix(sentence_matrix2)
    sentence_dict[remove_punctuation(sentence2)] = sentence_matrix2
        

In [ ]:
save_data_to_pkl(sentence_dict, 'OPPOxiaobu_sentence_dict.pkl')